In [1]:
import random
import string
p = 10000004873

def pick_random(prime):
    k = random.randint(2, prime - 1)
    return k 

def compute_powers_mod_p(k, p, n):
    powers = [1 for i in range(n)]
    for i in range(1, n):
        powers[i] = (powers[i - 1] * k) % p
    return powers

def replace_letters_with_numbers(s):
    letters = list(string.ascii_lowercase)
    letter_to_number = {letters[i] : i + 1 for i in range(len(letters))}
    replaced = [letter_to_number[c] for c in s]
    return replaced



def compute_hash(i, j, p, prefix_hashes, powers_mod_p):
    if i == 0:
        return prefix_hashes[j] 
    return (prefix_hashes[j] - powers_mod_p[j - i + 1] * prefix_hashes[i - 1]) % p



        



In [77]:
k = pick_random(p)
print(k)
s = "iwanttopartyintheusaaaaaaiwanttopartyyeahyeahyeah"
n = len(s)
powers = compute_powers_mod_p(k, p, n)
A = replace_letters_with_numbers(s)


9937073939


In [107]:
from functools import cmp_to_key
import string


class StringHasher:
    def __init__(self, s, p, k, powers_mod_p):
        self.s = s
        self.p = p
        self.k = k
        self.powers_mod_p = powers_mod_p
        self.A = self.replace_letters_with_numbers(self.s)
        self.prefix_hashes = self.compute_prefix_hashes(self.A, self.p, self.k)

    
    def replace_letters_with_numbers(self, s):
        letters = list(string.ascii_lowercase)
        letter_to_number = {letters[i] : i + 1 for i in range(len(letters))}
        replaced = [letter_to_number[c] for c in s]
        return replaced
    
    def compute_prefix_hashes(self, A, p, k):
        n = len(A)
        prefix_hashes = [0 for i in range(n)]
        prefix_hashes[0] = A[0]
        for i in range(1, n):
            prefix_hashes[i] = (self.k * prefix_hashes[i - 1] + A[i]) % self.p
        return prefix_hashes

    def compute_hash(self, i, j):
        if i == 0:
            return self.prefix_hashes[j] 
        return (self.prefix_hashes[j] - self.powers_mod_p[j - i + 1] * self.prefix_hashes[i - 1]) % self.p
    
    def indices_to_substring(self, i, j):
        return self.s[i : j + 1]
    
    def compare_two_substrings(self, indices1, indices2):
        return compare_two_substrings((indices1, self), (indices2, self))
    
    def hash_all_substrings(self):
        self.hash_to_indices = dict()
        self.unique_string_indices = []
        n = len(self.A)
        for i in range(n):
            for j in range(i, n):
                hash_value = self.compute_hash(i, j)
                if hash_value not in self.hash_to_indices:
                    self.hash_to_indices[hash_value] = ((i, j), self)
        
    

# Note: This function only works if the hashers in each argument were giving the same values of p and k.
def compare_two_substrings(indices_and_hasher1, indices_and_hasher2):
    (i1, j1), hasher1 = indices_and_hasher1
    (i2, j2), hasher2 = indices_and_hasher2
    # print('Comparing: ', hasher1.s[i1:j1 + 1], hasher2.s[i2:j2 + 1])

    diff1, diff2 = j1 - i1, j2 - i2
    min_diff = min(diff1, diff2)

    # So now, j2 - i2 >= j1 - i1.
    # We're going to binary search on the lengths of the strings to hash.
    # We'll find the shortest length for which the hashes are not equal, say l.
    # Then whichever is larger, A[i1 + l] or A[i2 + l], is from the "larger" string.
    max_range = min_diff
    min_range = 0
    curr_range = int((max_range + min_range) / 2)

    while True:
        # print("min_range, max_range, curr_range: ", min_range, max_range, curr_range)
        if max_range == min_range:
            if hasher1.A[i1 + curr_range] < hasher2.A[i2 + curr_range]:
                return -1
            elif hasher1.A[i1 + curr_range] > hasher2.A[i2 + curr_range]:
                return 1
            else:
                # In this case, the strings' hashes are equal for the stretch that we can compare them.
                # Whichever string is longer will be the lexicographically larger string.
                # E.e. we're comparing 'late' to 'later'
                if diff1 < diff2:
                    return -1
                elif diff1 > diff2:
                    return 1
                else:
                    return 0

        
        if hasher1.compute_hash(i1, i1 + curr_range) == hasher2.compute_hash(i2, i2 + curr_range):
            min_range = curr_range + 1
        else:
            max_range = curr_range
        curr_range = int((min_range + max_range) / 2)

In [88]:
def sort_all_unique_substrings(hashers):
    all_substring_indices = []
    unique_hashes = set()
    for hasher in hashers:
        hash_to_indices = hasher.hash_to_indices 
        indices = [hash_to_indices[hash] for hash in hash_to_indices if hash not in unique_hashes]
        all_substring_indices.extend(indices)
        unique_hashes.update([hash for hash in hash_to_indices])
    all_substring_indices.sort(key=cmp_to_key(compare_two_substrings))
    return all_substring_indices



def findStrings(w, queries):
    p = 10000004873
    k = pick_random(p)
    max_length = max(map(len, w))
    powers_mod_p = compute_powers_mod_p(k, p, max_length)
    hashers = []
    for word in w:
        hasher = StringHasher(word, p, k, powers_mod_p)
        hasher.hash_all_substrings()
        hashers.append(hasher)
    all_substrings_sorted = sort_all_unique_substrings(hashers)
    num_substrings = len(all_substrings_sorted)
    results = []
    for q in queries:
        if q > num_substrings:
            results.append('INVALID')
        else:
            ((i, j), hasher) = all_substrings_sorted[q - 1]
            results.append(hasher.indices_to_substring(i, j))
    return results



In [103]:
def substring_indices_to_substrings(substring_indices):
    substrings = []
    for i in range(len(substring_indices)):
        (i, j), hasher = substring_indices[i]
        substrings.append(hasher.indices_to_substring(i, j))
    return substrings

def testCode(w, queries = None):
    p = 10000004873
    k = pick_random(p)
    max_length = max(map(len, w))
    powers_mod_p = compute_powers_mod_p(k, p, max_length)
    hashers = []
    for word in w:
        hasher = StringHasher(word, p, k, powers_mod_p)
        hasher.hash_all_substrings()
        hashers.append(hasher)
    all_substrings_sorted = sort_all_unique_substrings(hashers)
    substrings = substring_indices_to_substrings(all_substrings_sorted)
    for s in substrings:
        print(s)


In [101]:
w1 = 'wptmevbtoaqywyzsnsnqjlztyusvieakuyrczfzguyztbpinykjzjwwxyuls'
w2 = 'cayqhteghiunxljkkozrloedzcrugskwamhkpvgmafbhdrmisubnejmmfzzz'
w = [w1, w2]

p = 10000004873
k = pick_random(p)
max_length = max(map(len, w))
powers_mod_p = compute_powers_mod_p(k, p, max_length)



hasher1 = StringHasher(w1, p, k, powers_mod_p)
hasher2 = StringHasher(w2, p, k, powers_mod_p)
compare_two_substrings(((9, 11), hasher1), ((1,3), hasher2))


Comparing:  aqy ayq
min_range, max_range, curr_range:  0 2 1
Hashes unequal
min_range, max_range, curr_range:  0 1 0
Hashes equal
min_range, max_range, curr_range:  1 1 1


-1

In [67]:
words = ['aab', 'aac', 'a', 'xyffgab']
queries = [2, 3, 8, 23, 30]

results = findStrings(words, queries)
print(results)

['aa', 'aab', 'c', 'xyff', 'yffg']


In [89]:
def substring_indices_to_substrings(indices):
    substrings = []
    for i in range(len(indices)):
        ((i, j), hasher) = indices[i]
        substrings.append(hasher.indices_to_substring(i, j))
    return substrings

def parse_input(filepath):
    with open(filepath, 'r') as f:
        n = int(f.readline())
        words = []
        while len(words) < n:
            words.append(f.readline().replace('\n', ''))
        q = int(f.readline())
        queries = []
        while len(queries) < q:
            queries.append(int(f.readline()))
    return words, queries

            


In [109]:
w, queries = parse_input('/tmp/test3.txt')
answers = findStrings(w, queries)
for a in answers:
    print(a)
# testCode(w)


adeuswpuxioizkdmxliepghmtxktqtnkicfxjoomxjqrecjwhinbgqxdictufugeqtlfwzfoodpdhjpmjmtfwcakwtjxniwmvdnequgckrrojkwqaxqzuzbhslkatipycnquhlwiqiptgyvsifkwbuozpacwanshqhplepdkdllprfwfwdgeztfwwxwfqtroocwgokgcnxdsjvknaaexhiaadgjbinccnhjgniasgtkzwzifikxcwyryghosli
akguvgqvvfndawlihsawqjpezumwvygzvfeplxeinperrbiewkhfpjysvlybagygdjraetzdpzrmqllvyfjetpismcfhapodbdivxcdyqskecadwakteqkklbxiyiqjipp
adeuswpuxioizkdmxliepghmtxktqtnkicfxjoomxjqrecjwhinbgqxdictufugeqtlfwzfoodpdhjpmjmtfwcakwtjxniwmvdnequgckrrojkwqaxqzuzbhslkatipycnquhlwiqiptgyvsifkwbuozpacwanshqhplepdkdllprfwfwdgeztfwwxwfqtroocwgokgcnxdsjvknaaexhiaadgjbinccnhjgniasgtkzwzifikxcwyryghosliqrvfjfxmxvlkxmqurehonsnpknauuhrjidvmweshexhyjkcnmibluewvfdnmjntagzkbxzlbtetqawumnja
afmllwsdrzcvqsfknxvnippnyrjigpwpgmymyvomkmxoulquqciwsezdbokzybbwkxkxhsegeuxunspblheztbussfkksiblcorhuovjhuafukwitzvxodqwuzbeambzfcjknooctfhagkqkeavvooorcppkbzgkyhmrutbclcrksphmdvkypvrumuzzkbtioankndpodsytgemehfdymbaytklkoowzoqltjxvctgtkqttargucrzygu
alygmdydgtglfnbkmtnwsveqjm

In [47]:
string_hasher = StringHasher(s, p, k, powers)

string_hasher.hash_all_substrings()
print(string_hasher.unique_string_indices[:10])

[((0, 0), <__main__.StringHasher object at 0x108da1610>), ((0, 1), <__main__.StringHasher object at 0x108da1610>), ((0, 2), <__main__.StringHasher object at 0x108da1610>), ((0, 3), <__main__.StringHasher object at 0x108da1610>), ((0, 4), <__main__.StringHasher object at 0x108da1610>), ((0, 5), <__main__.StringHasher object at 0x108da1610>), ((0, 6), <__main__.StringHasher object at 0x108da1610>), ((0, 7), <__main__.StringHasher object at 0x108da1610>), ((0, 8), <__main__.StringHasher object at 0x108da1610>), ((0, 9), <__main__.StringHasher object at 0x108da1610>)]


In [41]:
a = [string_hasher, string_hasher]
a[0].s = 'foo'
print(a[1].s)

foo


In [54]:
d = {'a': 1, 'b': 2, 'c': 3}
keys = set(['a', 'e'])
print([d[k] for k in d if k not in keys])
keys.update(['a', 'g', 'ff'])
print(keys)

[2, 3]
{'g', 'ff', 'e', 'a'}
